**Do not forget your TODO items**

----------------------

In [117]:
import math
import numpy
import numpy as np
from itertools import chain, tee
import functools

## Initialize hyperparameters

In [118]:
from us_city import load_us_cities
cities, norm_cities = load_us_cities()

DIM = 2 # dimension of the points
N = cities.shape[0]  # number of cities
M = int(2.5 * N)  # number of intermediate points between the cities

KAPPA_START = 0.2  # starting value of kappa
GAMMA = 1.05  # kappa damping factor

BETA = 10 # inverse temperature

NUM_ITERS = 6703 #10000 # number of iterations of the algorithm

PERTURBATION_RADIUS = 0.1 # perturbation radius for generating the intermediate points

# TODO: Delete the two parameters below
# Elastic Net parameters
ELASTIC_ALPHA = 0.2
ELASTIC_BETA = 2.0

In [119]:
class ElasticNet(object):
    
    def __init__(self):
        self.cities = norm_cities
        self.init_neurons()
        
        self.kappa = KAPPA_START


    def init_neurons(self):
        # TODO: Rewrite
        theta = np.linspace(0, 2 * math.pi, M, False)
        centroid = self.cities.mean(axis=0)

        self.neurons = np.vstack((np.cos(theta), np.sin(theta)))
        self.neurons *= PERTURBATION_RADIUS
        self.neurons += centroid[:, np.newaxis]
        self.neurons = self.neurons.transpose()


    def solve(self):
        for i in range(NUM_ITERS):
            self.update_weights()
            self.update_kappa()
            self.update_neurons_anand()
            if self.worst_dist < 0.01:
                break


    def update_kappa(self):
        self.kappa = max(self.kappa / GAMMA, 0.01)


    def update_weights_2(self):
        def softmax(x):
            """Compute softmax values for each sets of scores in x."""
            e_x = np.exp(x - np.max(x))
            return e_x / e_x.sum()
        # Calculate distances between the cities and the neurons
        self.diff = self.cities[:, np.newaxis] - self.neurons
        self.dist_squared = np.sum(self.diff ** 2, axis=-1)

        # self.weights = np.exp(-self.dist_squared / (2 * (self._k ** 2)))
        # self.weights /= self._weights.sum(axis=1)[:,numpy.newaxis]

        self.weights = softmax(BETA * np.sqrt(self.dist_squared))


    def _length_force(self):
        return np.array([
                self.neurons[(i + 1) % M]
                - 2 * self.neurons[i]
                + self.neurons[(i - 1) % M]
             for i in range(M)])

    def _length_force_2(self):
        return numpy.concatenate((
            [self.neurons[1] - 2 * self.neurons[0] 
             + self.neurons[M - 1]],
            
            [(self.neurons[i+1]
              - 2 * self.neurons[i]
              + self.neurons[i-1])
             for i in range(1, M - 1)],
            
            [self.neurons[0]
             - 2 * self.neurons[M - 1]
             + self.neurons[M - 2]]))


    def _dist_force(self):
        return np.array(
            [np.dot(self.weights[:, i],
                       self.diff[:, i]) for i in range(M)])


    def update_weights(self):
        """Compute w_ij, i = 1, 2, ..., |Cities|; j = 1, 2, ...., |Neurons|"""
        
        self.diff = self.cities[:,np.newaxis] - self.neurons
        self.dist_squared = np.sum(self.diff ** 2, axis=-1)

        self.worst_dist = np.max(np.min(np.sqrt(self.dist_squared), axis=1))

        self.weights = np.exp(-self.dist_squared / (2 * (self.kappa ** 2)))

        # At this point
        # self._weights[i,j] == unnormalized weight associated to city
        # i and neuron j

        self.weights /= self.weights.sum(axis=1)[:,np.newaxis]

        # At this point
        # self._weights[i,j] == normalized weight associated to city i
        # and neuron j

    def get_D(self):
        def d_ij(i, j):            
            if i == j:
                return np.sum(self.weights[:, int(j)])
            return 0
        return np.fromfunction(np.vectorize(d_ij), (M, M))


    @functools.cached_property
    def L(self):
        def L_ij(i, j, M):
            if i == j:
                return 2
            if i == (j + 1) % M:
                return 1
            if j == (i + 1) % M:
                return 1
            return 0
        return np.fromfunction(np.vectorize(L_ij), (M, M), M=M, dtype=int)


    def update_neurons_anand(self):
        self.neurons = np.linalg.inv((self.kappa * self.L + self.get_D())) @ self.weights.T @ self.cities


    def update_neurons(self):
        dist_force = self._dist_force()
        length_force = self._length_force()

        self.neurons += ELASTIC_ALPHA * dist_force \
            + ELASTIC_BETA * self.kappa * length_force

In [120]:
net = ElasticNet()
net.solve()

In [121]:
net.weights

array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 5.83608291e-181, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 6.49516168e-076, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       ...,
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 1.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000]])

In [122]:
net.worst_dist

0.2896716724505031

In [123]:
net.neurons

array([[ 4.86754485e-01,  3.59790530e-01],
       [-6.18160157e-01, -1.99623024e-01],
       [ 7.49565828e-01,  3.94555176e-02],
       [-8.37554342e-01, -1.66303190e-01],
       [ 9.25542855e-01,  2.93150862e-01],
       [-8.98610151e-01, -5.60379194e-01],
       [ 8.71677447e-01,  8.27607526e-01],
       [-5.72357217e-01, -2.64452717e-01],
       [ 2.73036987e-01, -2.98702092e-01],
       [ 2.62832432e-02,  8.61856901e-01],
       [-2.66965585e-02, -6.25660166e-01],
       [ 2.71098738e-02,  3.89463432e-01],
       [-9.30699451e-03, -1.31980406e-01],
       [-8.49588481e-03, -1.25502621e-01],
       [ 2.62987641e-02,  3.82985647e-01],
       [-1.32797656e-02, -4.89210403e-01],
       [ 2.60767119e-04,  5.95435158e-01],
       [-2.65124873e-04, -5.95491301e-01],
       [ 2.69482627e-04,  5.95547444e-01],
       [-1.37634728e-02, -4.94811204e-01],
       [ 2.72574630e-02,  3.94074963e-01],
       [-2.75128361e-02, -3.90814216e-01],
       [ 2.77682093e-02,  3.87553468e-01],
       [-2.

In [124]:
def decode_solution(dist2):
    """Return the permutation associated to the elastic."""

    neuron_city_pair = []

    for i in range(dist2.shape[0]):
        city = dist2.min(axis=1).argmin()
        neuron = dist2[city].argmin()

        dist2[city] = numpy.inf
        dist2[:,neuron] = numpy.inf

        neuron_city_pair.append((neuron, city))

    neuron_city_pair.sort(key=lambda x: x[0])
    return [x[1] for x in neuron_city_pair]


In [125]:
permutation = decode_solution(net.dist_squared)
permutation

[14, 20, 15, 7, 4, 13, 1, 3, 11, 8, 10, 5, 2, 19, 9, 18, 21, 12, 0, 17, 16, 6]

In [126]:
def _pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def compute_solution(permutation, original_cities):
    edges = list(_pairwise(chain(permutation, [permutation[0]])))

    length = sum([distance(original_cities[src_dst[0]],
                           original_cities[src_dst[1]]) for src_dst in edges])

    return (length, edges)

def distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return int(math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) + 0.5)

In [127]:
compute_solution(permutation, cities)

(688,
 [(14, 20),
  (20, 15),
  (15, 7),
  (7, 4),
  (4, 13),
  (13, 1),
  (1, 3),
  (3, 11),
  (11, 8),
  (8, 10),
  (10, 5),
  (5, 2),
  (2, 19),
  (19, 9),
  (9, 18),
  (18, 21),
  (21, 12),
  (12, 0),
  (0, 17),
  (17, 16),
  (16, 6),
  (6, 14)])